---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [30]:
def graph_identification():
    
    # Your Code Here
    
    # PA - power law - few with high degrees, standard deviation of num degree large
    # SW_L - path length small, clustering coefficient high, standard deviation of num degree small
    # SW_H - path length small, clustering coefficient low, standard deviation of num degree small
    answer = 'PA SW_L SW_L PA SW_H'
    
    return answer.split(' ')# Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [6]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [31]:
# testing the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# degree
degrees = nx.degree(G)
nx.set_node_attributes(G, 'degree', degrees)

# closeness centrality
closeness = nx.closeness_centrality(G)
nx.set_node_attributes(G, 'closeness', closeness)

# betweenness centrality
betweenness = nx.betweenness_centrality(G)
nx.set_node_attributes(G, 'betweenness', betweenness)

feats = ['degree', 'closeness', 'betweenness']
X = pd.DataFrame(G.node).T[feats].values
y = nx.get_node_attributes(G, 'ManagementSalary')
_, y = zip(*sorted(y.items(), key=lambda x: x[0]))

tr_ind = np.where(np.array(y) <= 1.0)[0]
te_ind = np.where(np.isnan(y))[0]

X_tr, X_val, y_tr, y_val = train_test_split(X[tr_ind], np.array(y)[tr_ind], train_size=0.7)
print(X_tr.shape, X_val.shape)
print(y_tr.shape, y_val.shape)
lr = LogisticRegression()
lr.fit(X_tr, y_tr)
score = lr.predict_proba(X_val)
predictions = lr.predict(X_val)
auc = roc_auc_score(y_val, score[:,1])
acc = accuracy_score(y_val, predictions)
print('AUC =', auc)
print('ACC =', acc)

(527, 3) (226, 3)
(527,) (226,)
AUC = 0.828247480403
ACC = 0.880530973451


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: invalid value encountered in less_equal


In [49]:
def salary_predictions():
    
    # Your Code Here
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    
    # degree
    degrees = nx.degree(G)
    nx.set_node_attributes(G, 'degree', degrees)

    # closeness centrality
    closeness = nx.closeness_centrality(G)
    nx.set_node_attributes(G, 'closeness', closeness)

    # betweenness centrality
    betweenness = nx.betweenness_centrality(G)
    nx.set_node_attributes(G, 'betweenness', betweenness)

    feats = ['degree', 'closeness', 'betweenness']
    df = pd.DataFrame(G.node).T[feats]
    X = df.values
    y = nx.get_node_attributes(G, 'ManagementSalary')
    _, y = zip(*sorted(y.items(), key=lambda x: x[0]))

    tr_ind = np.where(np.array(y) <= 1.0)[0]
    te_ind = np.where(np.isnan(y))[0]

    X_tr, y_tr = X[tr_ind], np.array(y)[tr_ind]
    X_te = X[te_ind]
    
    lr = LogisticRegression()
    lr.fit(X_tr, y_tr)
    
    df['prob'] = lr.predict_proba(X)[:, 1]
    
    return df.loc[te_ind, 'prob']# Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [20]:
# testing

# split to training and testing dataset
tr_ind = np.where(future_connections['Future Connection'] <= 1)
te_ind = np.where(np.isnan(future_connections['Future Connection']))

# the 7 connection attributes
# number of common neighbour
future_connections['common_nb'] = future_connections.index.to_series().map(lambda x: len(list(nx.common_neighbors(G
                                                                                                                  , x[0]
                                                                                                                  , x[1]))))
# Jaccard Coeff
future_connections['jc_coeff'] = future_connections.index.to_series().map(lambda x: list(nx.jaccard_coefficient(G, [x]))[0][2])

# resource allocation
future_connections['resource_allo'] = future_connections.index.to_series().map(lambda x: list(nx.resource_allocation_index(G, [x]))[0][2])

# Adamix Adar index
future_connections['adamic_adar'] = future_connections.index.to_series().map(lambda x: list(nx.adamic_adar_index(G, [x]))[0][2])

# preferential attachment
future_connections['pref_attach'] = future_connections.index.to_series().map(lambda x: list(nx.preferential_attachment(G, [x]))[0][2])
mean = future_connections['pref_attach'].mean()
std = future_connections['pref_attach'].std()
future_connections['pref_attach_norm'] = (future_connections['pref_attach'] - mean)/std

feats = ['common_nb', 'jc_coeff', 'resource_allo', 'adamic_adar', 'pref_attach_norm']

y = future_connections['Future Connection'].values
X = future_connections[feats].values

X_te, y_te = X[te_ind], y[te_ind]

X_tr, X_val, y_tr, y_val = train_test_split(X[tr_ind], y[tr_ind], train_size=0.7)

lr = LogisticRegression()
lr.fit(X_tr, y_tr)

score = lr.predict_proba(X_val)
auc = roc_auc_score(y_val, score[:,1])
print(auc)

In [29]:
def new_connections_predictions():
    
    # Your Code Here
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    
    # split to training and testing dataset
    tr_ind = np.where(future_connections['Future Connection'] <= 1)
    te_ind = np.where(np.isnan(future_connections['Future Connection']))

    # the 7 connection attributes
    # number of common neighbour
    future_connections['common_nb'] = future_connections.index.to_series().map(lambda x: len(list(nx.common_neighbors(G
                                                                                                                      , x[0]
                                                                                                                      , x[1]))))
    # Jaccard Coeff
    future_connections['jc_coeff'] = future_connections.index.to_series().map(lambda x: list(nx.jaccard_coefficient(G, [x]))[0][2])

    # resource allocation
    future_connections['resource_allo'] = future_connections.index.to_series().map(lambda x: list(nx.resource_allocation_index(G, [x]))[0][2])

    # Adamix Adar index
    future_connections['adamic_adar'] = future_connections.index.to_series().map(lambda x: list(nx.adamic_adar_index(G, [x]))[0][2])

    # preferential attachment
    future_connections['pref_attach'] = future_connections.index.to_series().map(lambda x: list(nx.preferential_attachment(G, [x]))[0][2])
    mean = future_connections['pref_attach'].mean()
    std = future_connections['pref_attach'].std()
    future_connections['pref_attach_norm'] = (future_connections['pref_attach'] - mean)/std

    feats = ['common_nb', 'jc_coeff', 'resource_allo', 'adamic_adar', 'pref_attach_norm']

    y = future_connections['Future Connection'].values
    X = future_connections[feats].values

    X_te = X[te_ind]  
    X_tr, y_tr = X[tr_ind], y[tr_ind]
    
    lr = LogisticRegression()
    lr.fit(X_tr, y_tr)

    df = future_connections.iloc[te_ind]
    df['score'] = lr.predict_proba(X_te)[:,1]

    return df['score']# Your Answer Here